In [6]:

import random
from neo4j import GraphDatabase
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
from langchain_openai import ChatOpenAI
from typing import Any
import os
from pydantic import PrivateAttr

In [8]:
NEO4J_URI = "<ENDPOINT_URI>"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "PASSWORD/API_KEY"

In [10]:
openai_api_key = "API_KEY"

os.environ["OPENAI_API_KEY"] = openai_api_key

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [6]:
"""
Random Synthetic data for Neo4j
"""
NUM_MEMBERS = 600
NUM_APPEALS = 1200

random.seed(42)

# Latent communities (DO NOT store explicitly in graph)
COMMUNITIES = {
    "C1": {"providers": range(1, 21), "denial": "DX001", "overturn_p": 0.80},
    "C2": {"providers": range(21, 41), "denial": "DX002", "overturn_p": 0.20},
    "C3": {"providers": range(41, 61), "denial": "DX003", "overturn_p": 0.70},
    "C4": {"providers": range(61, 81), "denial": "DX004", "overturn_p": 0.15},
}


CLEAR_DB = """
MATCH (n)
DETACH DELETE n
"""

CREATE_APPEAL = """
MERGE (m:Member {id:$member})
MERGE (p:Provider {id:$provider})
MERGE (d:DenialCode {code:$denial})
CREATE (a:Appeal {
    id: $appeal_id,
    outcome: $outcome,
    amount: $amount
})
MERGE (m)-[:FILED]->(a)
MERGE (p)-[:SUBMITTED]->(a)
MERGE (a)-[:DENIED_FOR]->(d)
MERGE (m)-[:VISITED]->(p)
"""

def generate_appeals():
    appeals = []

    for i in range(1, NUM_APPEALS + 1):
        community = random.choice(list(COMMUNITIES.values()))

        provider = f"P{random.choice(list(community['providers']))}"
        member = f"M{random.randint(1, NUM_MEMBERS)}"
        denial = community["denial"]

        outcome = (
            "OVERTURNED"
            if random.random() < community["overturn_p"]
            else "UPHELD"
        )

        appeals.append({
            "appeal_id": f"A{i}",
            "member": member,
            "provider": provider,
            "denial": denial,
            "outcome": outcome,
            "amount": random.randint(1_000, 25_000)
        })

    return appeals

In [8]:
driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USER, NEO4J_PASSWORD)
)

appeals = generate_appeals()

with driver.session() as session:
    print("🔹 Clearing existing graph...")
    session.run(CLEAR_DB)

    print("🔹 Ingesting appeals...")
    for a in appeals:
        session.run(CREATE_APPEAL, **a)

driver.close()

print("✅ Graph creation complete")
print(f"   Members: {NUM_MEMBERS}")
print(f"   Appeals: {NUM_APPEALS}")
print(f"   Providers: ~80")
print(f"   DenialCodes: {len(COMMUNITIES)}")

🔹 Clearing existing graph...
🔹 Ingesting appeals...
✅ Graph creation complete
   Members: 600
   Appeals: 1200
   Providers: ~80
   DenialCodes: 4


In [12]:
# Test
APPEAL_ID = "A1"

In [25]:
class Neo4jQueryTool(BaseTool):
    name: str = "Neo4jQueryTool"
    description: str = (
        "Query Neo4j healthcare appeals graph to retrieve "
        "appeal details and community statistics."
    )

    _driver: Any = PrivateAttr()

    def __init__(self):
        super().__init__()
        self._driver = GraphDatabase.driver(
            NEO4J_URI,
            auth=(NEO4J_USER, NEO4J_PASSWORD)
        )

    def _run(self, query: str) -> Any:
        with self._driver.session() as session:
            return session.run(query).data()

# Requires GDS capabilities
class GetAppealCommunities(BaseTool):
    name: str = "GetAppealCommunities"
    description: str = (
        "Get Louvain and Leiden community IDs for a given appeal ID."
    )

    _driver: Any = PrivateAttr()

    def __init__(self):
        super().__init__()
        self._driver = GraphDatabase.driver(
            NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD)
        )

    def _run(self, appeal_id: str):
        query = """
        MATCH (a:Appeal {id:$id})
        RETURN
            a.louvainCommunity AS louvain,
            a.leidenCommunity AS leiden
        """
        with self._driver.session() as session:
            return session.run(query, {"id": appeal_id}).data()

# Requires GDS capabilities
class GetCommunityStats(BaseTool):
    name: str = "GetCommunityStats"
    description: str = (
        "Get overturn statistics for a community using Louvain or Leiden."
    )

    _driver: Any = PrivateAttr()

    def __init__(self):
        super().__init__()
        self._driver = GraphDatabase.driver(
            NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD)
        )

    def _run(self, algorithm: str, community_id: int):
        if algorithm not in ["louvain", "leiden"]:
            raise ValueError("algorithm must be 'louvain' or 'leiden'")

        prop = "louvainCommunity" if algorithm == "louvain" else "leidenCommunity"

        query = f"""
        MATCH (a:Appeal)
        WHERE a.{prop} = $cid
        RETURN
            count(*) AS total,
            sum(CASE WHEN a.outcome='OVERTURNED' THEN 1 ELSE 0 END) AS overturned,
            round(toFloat(overturned)/total,2) AS rate
        """
        with self._driver.session() as session:
            return session.run(query, {"cid": community_id}).data()

class GetAppealContext(BaseTool):
    name: str = "GetAppealContext"
    description: str = (
        "Get provider and denial-based community context for an appeal using cypher query without community algorithms."
    )

    _driver: Any = PrivateAttr()

    def __init__(self):
        super().__init__()
        self._driver = GraphDatabase.driver(
            NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD)
        )

    def _run(self, appeal_id: str):
        query = """
        MATCH (p:Provider)-[:SUBMITTED]->(a:Appeal {id:$id})
        MATCH (a)-[:DENIED_FOR]->(d:DenialCode)
        RETURN p.id AS provider, d.code AS denial_code
        """
        with self._driver.session() as session:
            return session.run(query, {"id": appeal_id}).data()

class GetCommunityOverturnStats(BaseTool):
    name: str = "GetCommunityOverturnStats"
    description: str = (
        "Get overturn statistics for a provider + denial-code community without any community algorithms."
    )

    _driver: Any = PrivateAttr()

    def __init__(self):
        super().__init__()
        self._driver = GraphDatabase.driver(
            NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD)
        )

    def _run(self, provider: str, denial_code: str):
        query = """
        MATCH (p:Provider {id:$provider})-[:SUBMITTED]->(a:Appeal)-[:DENIED_FOR]->(d:DenialCode {code:$denial}) RETURN
    count(*) AS total,
    sum(CASE WHEN a.outcome='OVERTURNED' THEN 1 ELSE 0 END) AS overturned,
    round(
        toFloat(sum(CASE WHEN a.outcome='OVERTURNED' THEN 1 ELSE 0 END)) /
        count(*),
        2
    ) AS rate
        """
        with self._driver.session() as session:
            return session.run(
                query,
                {"provider": provider, "denial": denial_code}
            ).data()

In [27]:
llm = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0.1
)

In [37]:
appeal_agent = Agent(
    role="Healthcare Appeal Decision Analyst",
    goal=(
        "Analyze an appeal by comparing it against historical "
        "Louvain and Leiden communities in Neo4j and recommend "
        "If community algorithms like Lauvain and Leiden don't work, use overturn statistics and provider and denial without any community."
        "whether overturn is common, rare, or ambiguous."
    ),
    backstory="""
IMPORTANT SCHEMA RULES:
- Appeal community properties are:
  - a.louvainCommunity
  - a.leidenCommunity
- DO NOT use snake_case property names.
- If a property is missing, explicitly state it instead of guessing.

You must always query these exact properties.
""",
    tools=[Neo4jQueryTool(),GetAppealContext(),GetCommunityOverturnStats()],
    llm=llm,
    verbose=True
)

In [38]:
task = Task(
    description=f"""
    Analyze appeal with ID = {APPEAL_ID}.

    Required reasoning steps:
    1. Retrieve the appeal's Louvain and Leiden community IDs.
    2. For each community, compute:
       - total appeals
       - overturned appeals
       - overturn rate
    3. Compare the two algorithms:
       - Do they agree?
       - Is overturn common (>70%), rare (<30%), or mixed?
    4. Based on the comparison, recommend:
       - Uphold
       - Overturn
       - Manual review
       - Outreach / education program
    """,
    expected_output="""
    A structured analysis containing:
    - Appeal Summary
    - Community Analysis
    - Algorithm Agreement
    - Recommendation
    - Rationale
    """,
    agent=appeal_agent
)

In [39]:
crew = Crew(
    agents=[appeal_agent],
    tasks=[task],
    verbose=True
)

In [43]:
result = crew.kickoff()
print("\n===== FINAL AGENT OUTPUT =====\n")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3c59eb20-2a49-4aa8-a96c-795999eb474f                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Healthcare Appeal Decision Analyst                                                                      │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Analyze appeal with ID = A1.                                                                               │
│                                                                                                                 │
│      Required reasoning steps:                                                                                  │
│      1. Retrieve the appeal's Louvain and Leiden community IDs.                                                 │
│      2. For each community, compute:                                                                            │
│         - total appeals                                                                                         │
│         - overturned appeals                                                                                    │
│         - overturn rate                                                                                         │
│      3. Compare the two algorithms:                                                                             │
│         - Do they agree?                                                                                        │
│         - Is overturn common (>70%), rare (<30%), or mixed?                                                     │
│      4. Based on the comparison, recommend:                                                                     │
│         - Uphold                                                                                                │
│         - Overturn                                                                                              │
│         - Manual review                                                                                         │
│         - Outreach / education program                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Healthcare Appeal Decision Analyst                                                                      │
│                                                                                                                 │
│  Thought: Thought: I need to retrieve the appeal's Louvain and Leiden community IDs first to proceed with the   │
│  analysis.                                                                                                      │
│                                                                                                                 │
│  Using Tool: Neo4jQueryTool                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Healthcare Appeal Decision Analyst                                                                      │
│                                                                                                                 │
│  Thought: Thought: I now have the provider and denial code for the appeal. I will retrieve the overturn         │
│  statistics for the provider and denial code community.                                                         │
│                                                                                                                 │
│  Using Tool: GetCommunityOverturnStats                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Healthcare Appeal Decision Analyst                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│  Appeal Summary:                                                                                                │
│  - Appeal ID: A1                                                                                                │
│  - Provider: P1                                                                                                 │
│  - Denial Code: DX001                                                                                           │
│                                                                                                                 │
│  Community Analysis:                                                                                            │
│  - Total Appeals: 13                                                                                            │
│  - Overturned Appeals: 11                                                                                       │
│  - Overturn Rate: 85%                                                                                           │
│                                                                                                                 │
│  Algorithm Agreement:                                                                                           │
│  - Since there are no Louvain or Leiden community IDs available, we cannot compare the two algorithms           │
│  directly. However, the overturn rate of 85% indicates a strong tendency towards overturning in this context.   │
│                                                                                                                 │
│  Recommendation:                                                                                                │
│  - Overturn                                                                                                     │
│                                                                                                                 │
│  Rationale:                                                                                                     │
│  - The overturn rate of 85% is significantly above the common threshold of 70%, indicating that overturning is  │
│  a frequent outcome for this provider and denial code. Given the high overturn rate, it is recommended to       │
│  overturn the denial in this case.                                                                              │
│  ```                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3c59eb20-2a49-4aa8-a96c-795999eb474f                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```                                                                                              │
│  Appeal Summary:                                                                                                │
│  - Appeal ID: A1                                                                                                │
│  - Provider: P1                                                                                                 │
│  - Denial Code: DX001                                                                                           │
│                                                                                                                 │
│  Community Analysis:                                                                                            │
│  - Total Appeals: 13                                                                                            │
│  - Overturned Appeals: 11                                                                                       │
│  - Overturn Rate: 85%                                                                                           │
│                                                                                                                 │
│  Algorithm Agreement:                                                                                           │
│  - Since there are no Louvain or Leiden community IDs available, we cannot compare the two algorithms           │
│  directly. However, the overturn rate of 85% indicates a strong tendency towards overturning in this context.   │
│                                                                                                                 │
│  Recommendation:                                                                                                │
│  - Overturn                                                                                                     │
│                                                                                                                 │
│  Rationale:                                                                                                     │
│  - The overturn rate of 85% is significantly above the common threshold of 70%, indicating that overturning is  │
│  a frequent outcome for this provider and denial code. Given the high overturn rate, it is recommended to       │
│  overturn the denial in this case.                                                                              │
│  ```                                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯